In [ ]:
import getpass
from datetime import datetime, timedelta
from tapipy.tapis import Tapis
from time import time_ns

In [ ]:
username = getpass.getpass(prompt = "Username: ", stream=None)
password = getpass.getpass(prompt = "Password: ", stream=None)

base_url = "https://training.tapis.io"

client = Tapis(
    base_url = base_url, 
    username = username,
    password = password
) 

#generate access token
client.get_tokens()

In [ ]:
client.access_token

In [ ]:
unique_id = f"{username}_{str(time_ns())[-10:]}"
unique_id

In [ ]:
project_id = f"smart_data_workshop_project_{unique_id}"
site_id = f"smart_data_workshop_site_{unique_id}"
inst_id = f"smart_data_workshop_instrument_{unique_id}"

In [ ]:
#define and create project
project = {
    "project_name": project_id,
    "description": f"A smart data workshop project for user {username}",
    "owner": username,
    "pi": username,
    "active": True,
    "metadata": {}
}
proj = client.streams.create_project(**project)
proj

In [ ]:
#define and create site
site = {
    "project_id": project_id,
    "request_body": [{
        "site_name": site_id,
        "site_id": site_id,
        "description": f"A smart data workshop site for user {username}",
        "latitude": 19.89,
        "longitude": 155.58,
        "elevation": 10,
        "metadata": {}
    }]
}
site = client.streams.create_site(**site)
site

In [ ]:
#define and create instrument
instrument = {
    "project_id": project_id,
    "site_id": site_id,
    "request_body": [{
        "inst_name": inst_id,
        "inst_id": inst_id,
        "inst_description": f"A smart data workshop instrument for user {username}",
        "metadata": {}
    }]
}
inst = client.streams.create_instrument(**instrument)
inst

In [ ]:
#define and create variables
variables = {
    "project_id": project_id,
    "site_id": site_id,
    "inst_id": inst_id,
    "request_body": [{
        "var_name": "rainfall",
        "var_id": "rainfall",
        "units": "mm",
        "shortname": "rf",
        "metadata": {}
    },
    {
        "var_name": "temperature",
        "var_id": "temperature",
        "units": "C",
        "shortname": "temp",
        "metadata": {}
    }]
}
streams_vars = client.streams.create_variable(**variables)
streams_vars

In [ ]:
#Write Measurements - For today 
from datetime import datetime
import random
from random import randint
from datetime import timedelta
variables = []
#generate 10 sensor records
for i in range(0, 100):
    time = datetime.now() + timedelta(minutes=i)
    time_s = time.isoformat() 
    variables.append({"temperature": randint(60, 89),
                        "rainfall": randint(10, 200),
                        "datetime": time_s
                        })
#write observations to measurements endpoint for our instrument
result = client.streams.create_measurement(inst_id=inst_id, vars=variables)
print(result)

In [ ]:
#Download measurments as CSV
result = client.streams.list_measurements(inst_id=inst_id,
                                                    project_id=project_id, 
                                                    site_id=site_id,
                                                    start_date='2021-01-01T00:00:00Z',
                                                    end_date='2025-12-30T22:19:25Z',
                                                    format='csv')
result

In [ ]:
#Read Measurements to Data Frame
import pandas as pd
from io import StringIO
input = StringIO(str(result,'utf-8'))
df = pd.read_csv(input)
df['datetime']=pd.to_datetime(df['time'])
df.set_index('datetime',inplace=True)
df.pop('time')
df

In [ ]:
# Plot Measurements in the DataFrame
import matplotlib.pyplot as plt
import matplotlib.dates as md
%matplotlib inline
xfmt = md.DateFormatter('%H:%M:%S')
df.plot(lw=1, colormap='jet', marker='.', 
        markersize=12, title='Timeseries Stream Output', rot=90).xaxis.set_major_formatter(xfmt)
plt.tight_layout()
plt.legend(loc='best')
plt.savefig('test.png')

In [ ]:
#Create a storage system in Tapis so we can upload and download data from our server
#NOTE you system id needs to be unique across Tapis- so add your last_name
system_id = "training_tutorial_vm_" + unique_id

system_config = {
    "id": system_id,
    "description": "VM storage",
    "host": "149.165.153.42",
    "systemType": "LINUX",
    "defaultAuthnMethod": "PASSWORD",
    "effectiveUserId": username,
    "rootDir": f"/home/{username}/",
    "canExec": False
}

system = client.systems.createSystem(**system_config)
system

In [ ]:
# Add login credentials so Tapis can access the system - NORMALLY WE USED SSH KEYS but for this tutorial we will utilize the password auth
#NOTE you need to update the systemId to what you used above
client.systems.createUserCredential(systemId=system_id,userName=username, password=password)

In [ ]:
#List the files in our home directory
#NOTE the system id needs to be updated
client.files.listFiles(systemId=system_id, path="/")

In [ ]:
#Create an Actor that we can pass to a channel to execute when a threshold triggers
#NOTE update the name and the system_id
my_actor = client.actors.create_actor(image="scleveland/test_actor:0.0.29",
                                     name="Plot Streams Data-1",
                                     description="Actor that plots streams measurements.",
                                     default_environment={"system_id": system_id, "destination_path": "/"})
                                     
                                     
my_actor

In [ ]:
#Create a Channel to check for our Temperature exceeding 100- then execute an Actor to generate a plot and upload to our system
#NOTE you have to change you channel id to a unique one - add your lastname
channel_id = f"demo.tapis.channel-{unique_id}"
channel = client.streams.create_channels(channel_id=channel_id, 
                            channel_name=channel_id, 
                            template_id="default_threshold",
                            triggers_with_actions=[
                                {"inst_ids":[inst_id],
                                "condition":{"key":inst_id+".temperature",
                                              "operator":">", 
                                              "val":100},
                                 "action":{"method":"ACTOR","actor_id" : my_actor.id,
                                           "message":"Instrument: exceeded Temperature threshold"}}])
channel

In [ ]:
client.streams.list_channels()

In [ ]:
#Write Measurements - to trigger our Channel
from datetime import datetime
import random
from random import randint
#create measurement to trigger channel
variables = [{"temperature": 230,
    "rainfall": 0,
    "datetime":datetime_now
}]
#write observations to measurements endpoint for our instrument
result = client.streams.create_measurement(inst_id=inst_id, vars=variables)
result

In [ ]:
#List the Alerts issued on our Channel
client.streams.list_alerts(channel_id=channel.channel_id)

In [ ]:
#Fetch the latest alert and assign to a variable
alert = client.streams.list_alerts(channel_id=channel.channel_id).alerts[0]

In [ ]:
#Fetch our Actor execution log
client.actors.get_execution_logs(actor_id=alert.actor_id, 
                                 execution_id=alert.execution_id)

In [ ]:
#View our files on our VM - we should see the new plot file
client.files.listFiles(systemId=system_id, path="/")

In [ ]:
#Lets download the file to view here in our notebook
fileb =  client.files.getContents(systemId=system_id,path='/plot_2022-03-04T23:10:10Z.png')
with open("download.png","wb") as f:
    f.write(fileb)
    f.close()

In [ ]:
channel_id=f"training.discord.demo.tapis.channel_{unique_id}"
discord_channel = client.streams.create_channels(channel_id=channel_id, 
                            channel_name=channel_id, 
                            template_id="default_threshold",
                            triggers_with_actions=[
                                {"inst_ids":[inst_id],
                                 "condition":{"key":inst_id+".rainfall",
                                              "operator":">", 
                                              "val":150},
                                 "action":{"method":"DISCORD","webhook_url" :"https://discordapp.com/api/webhooks/949475414785523784/aXA7xwx_Zi2PxPsfG8QvjAC48ts191PFS2wchO8dmcSoJUE8PrLlxWxkrfMII3AjPsgO",
                                           "message":"My Instrument exceeded Rainfall threshold val ${ r.value}"}}], _tapis_debug=True)
discord_channel


In [ ]:
#Write Measurements - to trigger our Discord Channel
variables = [{"temperature": 80,
    "rainfall": 151,
    "datetime":datetime_now
}]
#write observations to measurements endpoint for our instrument
result = client.streams.create_measurement(inst_id=inst_id, vars=variables)
result